In [1]:
# !pip install -r requirements.txt --quiet

In [2]:
!pip install tensorflow --quiet

In [3]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import load_model
from evaluation_utils import run_evaluation_pipeline

/tmp/ipykernel_221/3345060064.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Model Evaluation

In [4]:
run_evaluation_pipeline("stage_4", data_mode="val")

base_train shape:  (800, 5748)
base_val shape:  (100, 5748)
motion_train shape:  (800, 3136)
motion_val shape:  (100, 3136)


/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


ROI_train shape:  (800, 18222)
ROI_val shape:  (100, 18222)
8
testing model:  model_hidden_1_lr_0.0001_dropout_0.2_l2_0.001.keras
4/4 [==============================] - 0s 59ms/step
testing model:  model_hidden_2_lr_0.0001_dropout_0.4_l2_0.0001.keras


/work/brainvision_modelling/evaluation_utils.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_values], ignore_index=True)


4/4 [==============================] - 0s 93ms/step
testing model:  model_hidden_2_lr_0.0001_dropout_0.2_l2_0.0001.keras
4/4 [==============================] - 0s 95ms/step
testing model:  model_hidden_2_lr_0.0001_dropout_0.2_l2_0.001.keras
4/4 [==============================] - 1s 88ms/step
testing model:  model_hidden_1_lr_0.0001_dropout_0.4_l2_0.0001.keras
4/4 [==============================] - 0s 58ms/step
testing model:  model_hidden_2_lr_0.0001_dropout_0.4_l2_0.001.keras
4/4 [==============================] - 1s 93ms/step
testing model:  model_hidden_1_lr_0.0001_dropout_0.4_l2_0.001.keras
4/4 [==============================] - 0s 60ms/step
testing model:  model_hidden_1_lr_0.0001_dropout_0.2_l2_0.0001.keras
4/4 [==============================] - 0s 59ms/step
finished testing sub: sub01, ROI: WB
ROI_train shape:  (800, 232)
ROI_val shape:  (100, 232)
Execution for sub01 ended at V1
ROI_train shape:  (800, 21573)
ROI_val shape:  (100, 21573)
8
testing model:  model_hidden_2_lr_0.00

/work/brainvision_modelling/evaluation_utils.py:140: FutureWarning: The provided callable <function mean at 0x7f35a05cd3f0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  )["correlation_score"].agg(
/work/brainvision_modelling/evaluation_utils.py:146: FutureWarning: The provided callable <function mean at 0x7f35a05cd3f0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ])["correlation_score"].agg(


In [5]:
# show best models
stage_name = "stage_4"

file_path = os.path.join("test_sco"re, stage_name, "test_results_aggregated.csv")
results = pd.read_csv(file_path)
results.head()

best_models = results.loc[results.groupby('ROI')['correlation_score'].idxmax()]
best_models

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2642115589.py, line 4)